<a href="https://colab.research.google.com/github/mgmk2/TensorFlow_v2_MNIST_sample/blob/master/tensorflow_v2_MNIST_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TF2.0の準備

In [0]:
# 2019/10/6現在、デフォルトではver1.14.0が入っているので、ver2.0.0をインストール
!pip install tensorflow-gpu==2.0.0

In [2]:
# verチェック
import tensorflow as tf
tf.__version__

'2.0.0'

# ネットワーク定義 (Functional API)

In [0]:
import tensorflow as tf

#=========================================================================
# ネットワーク定義
#=========================================================================

input_shape = (28, 28, 1) # 入力のshape. 最初の次元（バッチサイズ）は除く.

# ネットワークの定義
# 入力層
x = tf.keras.layers.Input(input_shape)
# 畳み込み層1
h = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME')(x)
h = tf.keras.layers.ReLU()(h)
# 畳み込み層2
h = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME')(h)
h = tf.keras.layers.ReLU()(h)
# 畳み込み層3
h = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME')(h)
h = tf.keras.layers.ReLU()(h)
# 線形層
h = tf.keras.layers.Flatten()(h)
y = tf.keras.layers.Dense(10)(h)

# モデルの作成
model = tf.keras.Model(x, y)


# ネットワーク定義 (Custom Model)

In [0]:
import tensorflow as tf

#=========================================================================
# ネットワーク定義
#=========================================================================

input_shape = (28, 28, 1) # 入力のshape. 最初の次元（バッチサイズ）は除く.

class CustomModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super(CustomModel, self).__init__(**kwargs)

        # 畳み込み層1
        self.conv1 = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME')
        self.act1 = tf.keras.layers.ReLU()
        # 畳み込み層2
        self.conv2 = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME')
        self.act2 = tf.keras.layers.ReLU()
        # 畳み込み層3
        self.conv3 = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME')
        self.act3 = tf.keras.layers.ReLU()
        # 線形層
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(10)

    def call(self, inputs):
        h = self.conv1(inputs)
        h = self.act1(h)
        h = self.conv2(h)
        h = self.act2(h)
        h = self.conv3(h)
        h = self.act3(h)
        h = self.flatten(h)
        y = self.dense(h)
        return y

model = CustomModel()


# 学習ステップとコスト関数の設定

In [0]:
#=========================================================================
# 学習ステップの定義
#=========================================================================

optimizer = tf.optimizers.Adam(1.0e-4)
train_loss = tf.keras.metrics.Mean() # コスト記録用
train_acc = tf.keras.metrics.SparseCategoricalAccuracy() # 精度計算・記録用

@tf.function
def train_step(inputs):
    images, labels = inputs

    # tf.GtadientTapeブロックで入力からロスまで計算
    with tf.GradientTape() as tape:
        logits = model(images)
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels, logits)
        
    # gradientを計算
    # gradientを計算する重みをsourcesとして指定することが必須
    # keras.Modelを使っているとmodel.trainable_variablesで渡すことができて便利
    grad = tape.gradient(loss, sources=model.trainable_variables)

    # optimizerで重みを更新
    optimizer.apply_gradients(zip(grad, model.trainable_variables))

    # lossの値を記録
    train_loss.update_state(loss)
    # train_loss(loss) # このように単純に__call__しても良い

    # 精度を記録
    train_acc.update_state(labels, logits)
    # train_acc(labels, logits) # このように単純に__call__しても良い

# 学習を実行 (Dataset API)

In [3]:
import time
import numpy as np

#=========================================================================
# データセット
#=========================================================================

# データセットをロード
# 今回はMNIST
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train[..., np.newaxis].astype(np.float32)
Y_train = Y_train.astype(np.int32)
N = X_train.shape[0]

# tf.data.Dataset APIを使う
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
dataset = dataset.shuffle(buffer_size=N)
dataset = dataset.batch(batch_size, drop_remainder=True)

#=========================================================================
# Dataset APIで学習を実行
#=========================================================================

print('train with Dataset API.')

epochs = 10
for epoch in range(epochs):
    time_start = time.time()

    for images, labels in dataset: # 1step分のデータを取り出し
        train_step((images, labels)) # 1step分の学習を実行

    # 平均ロスと平均精度
    # Metric.result()メソッドで取り出せる
    epoch_loss = train_loss.result()
    epoch_acc = 100 * train_acc.result()

    # epochの結果を表示
    time_epoch = time.time() - time_start
    print('epoch: {:} loss: {:.4f} acc: {:.2f}% time: {:.2f}s'.format(
        epoch + 1, epoch_loss, epoch_acc, time_epoch))

    # epoch毎にリセットしないと累積していく
    train_loss.reset_states()
    train_acc.reset_states()

# 学習済みの重みを保存
model.save_weights('weights_{:}'.format(epochs))

train with Dataset API.
epoch: 1 loss: 0.2117 acc: 95.02% time: 22.54s
epoch: 2 loss: 0.0443 acc: 98.65% time: 20.80s
epoch: 3 loss: 0.0265 acc: 99.16% time: 20.71s
epoch: 4 loss: 0.0162 acc: 99.48% time: 20.66s
epoch: 5 loss: 0.0121 acc: 99.61% time: 20.65s
epoch: 6 loss: 0.0090 acc: 99.69% time: 20.67s
epoch: 7 loss: 0.0069 acc: 99.77% time: 20.69s
epoch: 8 loss: 0.0046 acc: 99.85% time: 20.68s
epoch: 9 loss: 0.0043 acc: 99.85% time: 20.68s
epoch: 10 loss: 0.0047 acc: 99.84% time: 20.67s


# 学習を実行 (numpy array)

In [3]:
import time
import numpy as np

#=========================================================================
# データセット
#=========================================================================

# データセットをロード
# 今回はMNIST
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train[..., np.newaxis].astype(np.float32)
Y_train = Y_train.astype(np.int32)
N = X_train.shape[0]

# tf.data.Dataset APIを使う
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
dataset = dataset.shuffle(buffer_size=N)
dataset = dataset.batch(batch_size, drop_remainder=True)

#=========================================================================
# numpy配列を直接与えて学習を実行
#=========================================================================

print('train with numpy array.')

epochs = 10
for epoch in range(epochs):
    time_start = time.time()

    # ここから違う========================================================

    indices = np.arange(N)
    np.random.shuffle(indices)
    for i in range(N // batch_size):
        # 1step分のデータを取り出し
        indices_batch = indices[i * batch_size:(i + 1) * batch_size]
        images = X_train[indices_batch]
        labels = Y_train[indices_batch]
        
        # 1step分の学習を実行
        # numpy配列を直接入れてOK
        train_step((images, labels))
    
    # ここまで違う========================================================
    
    # 平均ロスと平均精度
    # Metric.result()メソッドで取り出せる
    epoch_loss = train_loss.result()
    epoch_acc = 100 * train_acc.result()

    # epochの結果を表示
    time_epoch = time.time() - time_start
    print('epoch: {:} loss: {:.4f} acc: {:.2f}% time: {:.2f}s'.format(
        epoch + 1, epoch_loss, epoch_acc, time_epoch))

    # epoch毎にリセットしないと累積していく
    train_loss.reset_states()
    train_acc.reset_states()

# 学習済みの重みを保存
model.save_weights('weights_{:}'.format(epochs))

train with numpy array.
epoch: 1 loss: 0.1827 acc: 95.35% time: 20.78s
epoch: 2 loss: 0.0403 acc: 98.75% time: 19.05s
epoch: 3 loss: 0.0229 acc: 99.26% time: 18.99s
epoch: 4 loss: 0.0153 acc: 99.45% time: 19.01s
epoch: 5 loss: 0.0106 acc: 99.62% time: 18.94s
epoch: 6 loss: 0.0081 acc: 99.73% time: 18.92s
epoch: 7 loss: 0.0067 acc: 99.76% time: 18.95s
epoch: 8 loss: 0.0052 acc: 99.84% time: 18.93s
epoch: 9 loss: 0.0052 acc: 99.82% time: 18.88s
epoch: 10 loss: 0.0034 acc: 99.88% time: 18.95s
